<a href="https://colab.research.google.com/github/noeljaymon/Emotion-Detection-using-DL/blob/master/Inception_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
from time import sleep
import re
import os
import argparse
from collections import OrderedDict
import matplotlib.animation as animation

### Image processing ###
from scipy.ndimage import zoom
from scipy.spatial import distance
import imutils
from scipy import ndimage
import cv2
import dlib
from __future__ import division
from imutils import face_utils

### CNN models ###
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, SeparableConv2D
from keras.utils import np_utils
from keras.regularizers import l2#, activity_l2
from keras.optimizers import SGD, RMSprop
from keras.utils import to_categorical
from keras.layers.normalization import BatchNormalization
from keras import models
from keras.utils.vis_utils import plot_model
from keras.layers import Input, GlobalAveragePooling2D
from keras.models import Model
from tensorflow.keras import layers

Using TensorFlow backend.


In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm

### Same trained models ###
import h5py
from keras.models import model_from_json
import pickle

In [ ]:
dataset= pd.read_csv('/content/drive/My Drive/fer2013.csv')

In [ ]:
train = dataset[dataset["Usage"] == "Training"]

#Obtaining test data where usage is "PublicTest"
test = dataset[dataset["Usage"] == "PublicTest"]
test2=dataset[dataset["Usage"] == "PrivateTest"]

#Converting " " separated pixel values to list
train['pixels'] = train['pixels'].apply(lambda image_px : np.fromstring(image_px, sep = ' '))
test['pixels'] = test['pixels'].apply(lambda image_px : np.fromstring(image_px, sep = ' '))
test2['pixels'] = test2['pixels'].apply(lambda image_px : np.fromstring(image_px, sep = ' '))
dataset['pixels'] = dataset['pixels'].apply(lambda image_px : np.fromstring(image_px, sep = ' '))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

KeyboardInterrupt: ignored

In [ ]:
train.shape
shape_x = 48
shape_y = 48

In [ ]:
X_train = train.iloc[:, 1].values
y_train = train.iloc[:, 0].values

X_test = test.iloc[:, 1].values
y_test = test.iloc[:, 0].values

In [ ]:
X_test2 = test2.iloc[:, 1].values
y_test2 = test2.iloc[:, 0].values

In [ ]:
X = dataset.iloc[:,1].values
y = dataset.iloc[:,0].values

#np.vstack stack arrays in sequence vertically (picking element row wise)
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)
X = np.vstack(X)

In [ ]:
X_test2 = np.vstack(X_test2)

In [ ]:
X_train

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0],48,48,1))
y_train = np.reshape(y_train, (y_train.shape[0],1))

X_test = np.reshape(X_test, (X_test.shape[0],48,48,1))
y_test = np.reshape(y_test, (y_test.shape[0],1))

X = np.reshape(X, (X.shape[0],48,48,1))
y = np.reshape(y, (y.shape[0],1))

print("Shape of X_train and y_train is " + str(X_train.shape) +" and " + str(y_train.shape) +" respectively.")
print("Shape of X_test and y_test is " + str(X_test.shape) +" and " + str(y_test.shape) +" respectively.")

In [ ]:
X_test2 = np.reshape(X_test2, (X_test2.shape[0],48,48,1))

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_test2 = X_test2.astype('float32')
X = X.astype('float32')

# Scale the data to lie between 0 to 1
X_train /= 255
X_test /= 255
X_test2 /= 255
X /= 255

# Change the labels from integer to categorical data
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_test2 = to_categorical(y_test2)
y = to_categorical(y)

In [ ]:
local_path ='/content/drive/My Drive/emotion numpy files'

In [ ]:

#np.save( 'X_train', X_train)
#np.save( 'X_test', X_test)
#np.save( 'X', X)
#np.save( 'y_train', y_train)
#np.save( 'y_test', y_test)
#np.save( 'y', y)

In [ ]:
#np.save( 'X_test2', X_test2)
#np.save( 'y_test2', y_test2)

In [ ]:
local_path ='/content/drive/My Drive/emotion numpy files'
X_train = np.load(local_path + "/X_train.npy")
X_test2 = np.load(local_path + "/X_test2.npy")
y_train = np.load(local_path + "/y_train.npy")
y_test2 = np.load(local_path + "/y_test2.npy")
y_test = np.load(local_path + "/y_test.npy")
X_test = np.load(local_path + "/X_test.npy")

In [ ]:
shape_x = 48
shape_y = 48
nRows,nCols,nDims = X_train.shape[1:]
input_shape = (nRows, nCols, nDims)
classes = [0,1,2,3,4,5,6]
nClasses = len(classes)


In [ ]:
nClasses 

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
input_img = Input(shape=(shape_x, shape_y, 1))

layer1 = Conv2D(10, (3, 3), padding='same', activation='relu')(input_img)
layer1_2 = Conv2D(20, (3, 3), padding='same', activation='relu')(layer1)
layer2 = MaxPooling2D(pool_size=(3, 3))(layer1_2)
bn1 = BatchNormalization()(layer2)

layer3 = Conv2D(30, (3, 3), padding='same', activation='relu')(bn1)
layer3_2 = Conv2D(40, (3, 3), padding='same', activation='relu')(layer3)
layer4 = MaxPooling2D(pool_size=(3, 3))(layer3_2)
bn2 = BatchNormalization()(layer4)

layer5 = Conv2D(50, (3, 3), padding='same', activation='relu')(bn2)
layer5_2 = Conv2D(60, (3, 3), padding='same', activation='relu')(layer5)
layer6 = MaxPooling2D(pool_size=(3, 3))(layer5_2)
bn3 = BatchNormalization()(layer6)

Conv11 = Conv2D(1, (1, 1), padding='same', activation='relu')(bn3)
Conv33 = Conv2D(4, (3, 3), padding='same', activation='relu')(bn3)
Conv332 = Conv2D(1, (3, 3), padding='same', activation='relu')(Conv33)
Conv55 = Conv2D(4, (5, 5), padding='same', activation='relu')(bn3)
Conv552 = Conv2D(1, (3, 3), padding='same', activation='relu')(Conv55)
#Pool33 = MaxPooling2D(pool_size=(3, 3))(bn3)
#ConvPool1 = Conv2D(4, (1, 1), padding='same', activation='relu')(Pool33)

intermediate1 = keras.layers.concatenate([Conv11, Conv332, Conv552], axis=1)

In [ ]:
Conv2_11 = Conv2D(1, (1, 1), padding='same', activation='relu')(intermediate1)
Conv2_33 = Conv2D(4, (3, 3), padding='same', activation='relu')(intermediate1)
Conv2_332 = Conv2D(1, (3, 3), padding='same', activation='relu')(Conv2_33)
Conv2_55 = Conv2D(4, (5, 5), padding='same', activation='relu')(intermediate1)
Conv2_552 = Conv2D(1, (3, 3), padding='same', activation='relu')(Conv2_55)
Pool2_33 = MaxPooling2D(pool_size=(1, 1))(intermediate1)
ConvPool2 = Conv2D(1, (1, 1), padding='same', activation='relu')(Pool2_33)

intermediate2 = keras.layers.concatenate([Conv2_11, Conv2_332, Conv2_552,ConvPool2], axis=1)

Conv3_11 = Conv2D(1, (1, 1), padding='same', activation='relu')(intermediate2)
Conv3_33 = Conv2D(4, (3, 3), padding='same', activation='relu')(intermediate2)
Conv3_332 = Conv2D(1, (3, 3), padding='same', activation='relu')(Conv3_33)
Conv3_55 = Conv2D(4, (5, 5), padding='same', activation='relu')(intermediate2)
Conv3_552 = Conv2D(1, (3, 3), padding='same', activation='relu')(Conv3_55)
Pool3_33 = MaxPooling2D(pool_size=(1, 1))(intermediate2)
ConvPool3 = Conv2D(1, (1, 1), padding='same', activation='relu')(Pool3_33)

intermediate3 = keras.layers.concatenate([Conv3_11, Conv3_332, Conv3_552,ConvPool3], axis=1)
Conv4_11 = Conv2D(1, (1, 1), padding='same', activation='relu')(intermediate3)
Conv4_33 = Conv2D(4, (3, 3), padding='same', activation='relu')(intermediate3)
Conv4_332 = Conv2D(1, (3, 3), padding='same', activation='relu')(Conv4_33)
Conv4_55 = Conv2D(4, (5, 5), padding='same', activation='relu')(intermediate3)
Conv4_552 = Conv2D(1, (3, 3), padding='same', activation='relu')(Conv4_55)
#Pool3_33 = MaxPooling2D(pool_size=(3, 3))(intermediate3)
#ConvPool3 = Conv2D(4, (1, 1), padding='same', activation='relu')(Pool3_33)
intermediate4 = keras.layers.concatenate([Conv4_11, Conv4_332, Conv4_552], axis=1)
#Pool4 = MaxPooling2D(pool_size=(3, 3))(intermediate3)

Flat = Flatten()(intermediate4)

Dense1 = Dense(1024, activation='relu')(Flat)
drp=Dropout(0.4)(Dense1)
Dense2 = Dense(512, activation='relu')(drp)
Dense3 = Dense(7, activation='softmax')(Dense2)

In [ ]:
model = Model([input_img], Dense3)

In [ ]:
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
 
batch_size = 128
epochs = 150


In [ ]:
datagen = ImageDataGenerator(
        zoom_range=0.1,          # randomly zoom into images
        rotation_range=30,       # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,   # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,    # randomly flip images
        vertical_flip=False)     # randomly flip images

history = model.fit_generator(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=int(np.ceil(X_train.shape[0] / float(batch_size))),
    epochs = epochs, 
    validation_data=(X_test, y_test))

Epoch 1/150
225/225 [==============================] - 22s 100ms/step - loss: 1.7786 - accuracy: 0.2577 - val_loss: 1.9244 - val_accuracy: 0.2494
Epoch 2/150
225/225 [==============================] - 13s 56ms/step - loss: 1.6422 - accuracy: 0.3322 - val_loss: 1.7478 - val_accuracy: 0.2976
Epoch 3/150
225/225 [==============================] - 13s 58ms/step - loss: 1.5129 - accuracy: 0.3895 - val_loss: 1.4453 - val_accuracy: 0.4138
Epoch 4/150
225/225 [==============================] - 13s 57ms/step - loss: 1.4357 - accuracy: 0.4297 - val_loss: 1.4191 - val_accuracy: 0.4383
Epoch 5/150
225/225 [==============================] - 13s 57ms/step - loss: 1.3765 - accuracy: 0.4604 - val_loss: 1.4774 - val_accuracy: 0.4330
Epoch 6/150
225/225 [==============================] - 13s 57ms/step - loss: 1.3349 - accuracy: 0.4815 - val_loss: 1.3307 - val_accuracy: 0.4845
Epoch 7/150
225/225 [==============================] - 13s 58ms/step - loss: 1.3077 - accuracy: 0.4902 - val_loss: 1.3294 - val_a

In [ ]:

# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=2.0)
plt.plot(history.history['val_loss'],'b',linewidth=2.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'r',linewidth=2.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
score = model.evaluate(X_test2, y_test2, batch_size=batch_size)
print(score)

3589/3589 [==============================] - 0s 63us/step
[1.015868356215253, 0.6146559119224548]
